In [1]:
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
import random
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

In [4]:

class SeuAmbiente(gym.Env):
    def __init__(self, volume_inicial=0, relevancia_inicial=0, 
                 delta_volume=400, delta_relevancia=0.4):
        super().__init__()

        self.volume_inicial = volume_inicial
        self.relevancia_inicial = relevancia_inicial

        self.delta_volume_inicial = delta_volume
        self.delta_relevancia_inicial = delta_relevancia

        self.ativacoes = []
        self.media_ativacoes = 0
        
        # Define o espaço de ações:
        self.action_space = spaces.Box(
            low=np.array([-50, -0.1]),  # Limites mínimos para delta_volume e delta_relevancia
            high=np.array([50, 0.1]),   # Limites máximos
            dtype=np.float32
        )

        self.observation_space = spaces.Dict({
            'volume': spaces.Discrete(1001),  # 0 a 1000
            'relevancia': spaces.Discrete(100),  # 0 a 99 -> 0.00 a 0.99
            'delta_volume': spaces.Discrete(10001),  # 0 a 10000
            'delta_relevancia': spaces.Discrete(100),  # 0 a 99 -> 0.00 a 0.99
            'media_ativacoes': spaces.Discrete(201),  # 0 a 200 -> -1.0 a 1.0
        })


        self.reset()

    def reset(self, seed=None, options=None):
        """Reinicia o ambiente para um novo episódio."""
        super().reset(seed=seed) # Para garantir compatibilidade com versões futuras do Gymnasium

        self.volume_atual = self.volume_inicial  
        self.relevancia_atual = self.relevancia_inicial 

        self.delta_volume = self.delta_volume_inicial
        self.delta_relevancia = self.delta_relevancia_inicial

        self.ativacoes = []
        self.media_ativacoes = 0

        # Retorna a observação e informações adicionais (vazio neste caso)
        return self._get_obs(), {}

    def _get_obs(self):
        """Retorna o estado atual do ambiente."""
        return {'volume': int(self.volume_atual), 
                'relevancia': int(self.relevancia_atual * 100),
                'delta_volume': int(self.delta_volume),
                'delta_relevancia': int(self.delta_relevancia * 100),
                'media_ativacoes': int(self.media_ativacoes * 100)+100
                }

    def step(self, action):
        """Aplica a ação e atualiza o ambiente."""
        delta_volume, delta_relevancia = action[0], action[1]  # Extrai os valores do array

        # Atualiza os deltas (sem necessidade de ajustes adicionais)
        self.delta_relevancia += delta_relevancia
        self.delta_volume += delta_volume

        self.delta_relevancia = np.clip(self.delta_relevancia, 0.1, 0.99)
        self.delta_volume = np.clip(self.delta_volume, 200, 1000)

        # Atualiza o estado (ajuste a lógica conforme necessário)
        self.volume_atual += random.randint(22, 27) 
        self.relevancia_atual += random.uniform(0.28, 0.32)

        # Impede que os valores ultrapassem os limites
        self.volume_atual = np.clip(self.volume_atual, 0, 1000)
        self.relevancia_atual = np.clip(self.relevancia_atual, 0, 1)

        # Determina se o processo foi ativado
        processo_ativado = self.volume_atual >= self.delta_volume or \
                           self.relevancia_atual >= self.delta_relevancia
        
        # Reset caso o processo seja ativado
        if processo_ativado:
            print(f"Ativou: delta V: {self.delta_volume} delta R: {self.delta_relevancia}")

            if self.volume_atual >= self.delta_volume and self.relevancia_atual >= self.delta_relevancia:
                self.ativacoes.append(0)
            elif self.volume_atual >= self.delta_volume:
                self.ativacoes.append(1)
            elif self.relevancia_atual >= self.delta_relevancia:
                self.ativacoes.append(-1)
            

            if len(self.ativacoes) > 10:
                self.ativacoes.pop(0)

            if len(self.ativacoes) > 0:
                self.media_ativacoes = sum(self.ativacoes) / (len(self.ativacoes))

            print((self.media_ativacoes * 100) + 100)

            self.volume_atual = 0
            self.relevancia_atual = 0

        # Calcula a recompensa
        reward = self.calcular_recompensa(processo_ativado, 
                                          self.volume_atual, 
                                          self.relevancia_atual,
                                          self.delta_volume,
                                          self.delta_relevancia,
                                          self.media_ativacoes
                                          )

        done = False
        truncated = False  

        return self._get_obs(), float(reward), done, truncated, {}

    def calcular_recompensa(self, processo_ativado, volume, relevancia, delta_volume, delta_relevancia, media_ativacoes):
        recompensa = 0

        # Penalidades por deltas fora do intervalo
        if delta_volume < 200:
            recompensa -= (200 - delta_volume) / 100
        elif delta_volume > 800:
            recompensa -= (delta_volume - 800) / 100

        if delta_relevancia < 0.2:
            recompensa -= (0.2 - delta_relevancia) * 10 
        elif delta_relevancia > 0.7:
            recompensa -= (delta_relevancia - 0.7) * 10

        if abs(media_ativacoes) > 0.5:
            recompensa -= 0.8
        else:
            recompensa += 1

        # Bônus por deltas no intervalo ideal
        if 200 <= delta_volume <= 700 and 0.2 <= delta_relevancia <= 0.7:
            recompensa += 1
        

        return recompensa
        

In [5]:
# Criar uma instância do seu ambiente
env = SeuAmbiente()

# Verificar se o ambiente está definido corretamente (opcional, mas recomendado)
check_env(env)

# Criar o modelo A2C (ao invés de DQN)
model = PPO("MultiInputPolicy", env, verbose=1, gamma=0.4)

# Treinar o agente
model.learn(total_timesteps=10000)

# Salvar o modelo treinado
model.save("modelo_ppo10k_windows_(1)") 

# Executar o agente no ambiente e visualizar o desempenho
obs, _ = env.reset()  # Desempacota a tupla aqui
for i in range(3000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    print(action)
    print(reward)
    print(obs)
    if done:
        obs, _ = env.reset()

c:\Users\Vitor\ETA-IA\ETA-IA\.venv\Lib\site-packages\stable_baselines3\common\env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


Ativou: delta V: 414.9273319244385 delta R: 0.5096015200018883
0.0
Ativou: delta V: 394.13306999206543 delta R: 0.7038795199245215
0.0
Ativou: delta V: 315.003369808197 delta R: 0.8628842901438475
0.0
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Ativou: delta V: 398.7981358319521 delta R: 0.46260969489812853
0.0
Ativou: delta V: 399.0946618486196 delta R: 0.519022861123085
0.0
Ativou: delta V: 399.5752495098859 delta R: 0.519022861123085
0.0
Ativou: delta V: 400.6421256829053 delta R: 0.8190228655934334
0.0
Ativou: delta V: 402.9927779007703 delta R: 0.6596608139574528
0.0
Ativou: delta V: 400.9493811596185 delta R: 0.5596608124673367
0.0
Ativou: delta V: 400.4270861912519 delta R: 0.5596608124673367
0.0
Ativou: delta V: 401.94088933058083 delta R: 0.35966080948710444
0.0
Ativou: delta V: 404.3780712801963 delta R: 0.2596608079969883
0.0
Ativou: delta V: 404.1042248625308 delta R: 0.1596608065068722
0.0
Ativou: delta V: 402.6526796240359